#0.環境構築

##0-1.まずは以下のサイトよりDeepInfraのアカウントを作成してください。

https://deepinfra.com/

##0-2.以下よりAPI KEYを取得してセルにセットしてください。

https://deepinfra.com/dash/api_keys

In [ ]:
DEEPINFRA_API_KEY = "cHv2ftIYyvvzPIHatIJbQZ8ImSducJGg"

##0-3.Google Driveをマウント

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


エクセルファイルの保存先パスを設定してください。

In [ ]:
G_FOLDER = "/content/drive/MyDrive"

##0-4.以下のコードで必要なライブラリのインストールと、APIのテスト実行を行います。

In [ ]:
!pip install -qU openai datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.9 MB/s eta 0:00:00


In [ ]:
# Assume openai>=1.0.0
from openai import OpenAI

# Create an OpenAI client with your deepinfra token and endpoint
openai = OpenAI(
    api_key=DEEPINFRA_API_KEY,
    base_url="https://api.deepinfra.com/v1/openai",
)

In [ ]:
chat_completion = openai.chat.completions.create(
    model="mistralai/Mixtral-8x22B-Instruct-v0.1",
    messages=[{"role": "system", "content": "あなたは日本語で回答するAIボットです。"},
     {"role": "user", "content": "以下の文章が文法的に正しいかどうかを評価してください。文法的に正しい場合には1を、誤りを含んでいる場合には0を選択してください。  本件に関しましては、貴社のご要望を踏まえて最適なソリューションをご提案させていただきます次第でございます。"}],
)

print(chat_completion.choices[0].message.content)

 1


#1.データ生成処理の実行

##1-1.プロンプト準備（エンティティリンキング）

In [ ]:
PROMPT_TEMPLATE = """
以下のタスク指示例を参考に、10種類の多様なinputとoutputを考えてください。これらのタスク指示はGPTモデルに与えられ、その指示を完了するためのGPTモデルを評価します。

要件は以下の通りです：
1.与えられたテキストから固有表現を全て抽出します。
2.抽出した固有表現を知識ベース内で対応するエンティティにリンクします。
3.回答は「固有表現（種類, QID）」の形式で示してください。
4.与えられるテキストは一文としてください。
5.日本語だけを使った文章が求められます。
6.ビジネス文書やニュース記事の文章を想定しています。

多様性を与えるヒントは以下のとおりです。:
多様性を与えるヒントは以下のとおりです。:
1.文章の種類：ニュース記事、ビジネス文書、技術レポート、学術論文、ブログ記事、SNS投稿など、異なるジャンルの文章を含める。
2.エンティティの種類：人物（政治家、スポーツ選手、科学者など）、場所（都市、国、建物など）、組織（企業、団体、政府機関、プロジェクトなど）などを含める。
3.関係の種類：職業、役職、勤務地、目的、手段、期間、結果、条件など、さまざまな関係を網羅する。
4.長さと複雑さ：簡単な文章から複雑な文章まで、バリエーションを持たせる。
5.文体：フォーマル、インフォーマル、説明など、異なる文体の文章を含める。
6.時制：現在、過去、未来の時制に配慮した文章を含める。
7.言語の多様性：エンティティ関係が異なる言い回しで表現した文章を含める。

タスク指示例：
$FewShotExamples

回答フォーマット例(JSON):
[
  {
    "id": 1,
    "instruction": "以下の文章から、エンティティ間の関係を見つけてエンティティペアと関係の種類を示してください。",
    "input": "<入力文章>",
    "output": "<固有表現（種類, QID）>"
  },
  {
    "id": 2,
    "instruction": "以下の文章から、エンティティ間の関係を見つけてエンティティペアと関係の種類を示してください。",
    "input": "<入力文章>",
    "output": "<固有表現（種類, QID）>"
  },
  {
    "id": 3,
    "instruction": "以下の文章から、エンティティ間の関係を見つけてエンティティペアと関係の種類を示してください。",
    "input": "<入力文章>",
    "output": "<固有表現（種類, QID）>"
  },
  {
    "id": 4,
    "instruction": "以下の文章から、エンティティ間の関係を見つけてエンティティペアと関係の種類を示してください。",
    "input": "<入力文章>",
    "output": "<固有表現（種類, QID）>"
  },
  {
    "id": 5,
    "instruction": "以下の文章から、エンティティ間の関係を見つけてエンティティペアと関係の種類を示してください。",
    "input": "<入力文章>",
    "output": "<固有表現（種類, QID）>"
  },
  {
    "id": 6,
    "instruction": "以下の文章から、エンティティ間の関係を見つけてエンティティペアと関係の種類を示してください。",
    "input": "<入力文章>",
    "output": "<固有表現（種類, QID）>"
  },
  {
    "id": 7,
    "instruction": "以下の文章から、エンティティ間の関係を見つけてエンティティペアと関係の種類を示してください。",
    "input": "<入力文章>",
    "output": "<固有表現（種類, QID）>"
  },
  {
    "id": 8,
    "instruction": "以下の文章から、エンティティ間の関係を見つけてエンティティペアと関係の種類を示してください。",
    "input": "<入力文章>",
    "output": "<固有表現（種類, QID）>"
  },
  {
    "id": 9,
    "instruction": "以下の文章から、エンティティ間の関係を見つけてエンティティペアと関係の種類を示してください。",
    "input": "<入力文章>",
    "output": "<固有表現（種類, QID）>"
  },
  {
    "id": 10,
    "instruction": "以下の文章から、エンティティ間の関係を見つけてエンティティペアと関係の種類を示してください。",
    "input": "<入力文章>",
    "output": "<固有表現（種類, QID）>"
  }
]
"""

In [ ]:
FEWSHOT_EXAMPLES = [
 {
    "id": 1,
    "instruction": "与えられたテキストから固有表現（組織名、人名、地名、固有物名、日付表現、時刻表現、金額表現、割合表現）をすべて抽出し、知識ベース内の該当エンティティにリンクしてください。回答は「固有表現（種類, 知識ベースエンティティ）」の形式で示してください。",
    "input": "豊田章男はトヨタ自動車の代表取締役会長であり、愛知県豊田市に本社を置いています。",
    "output": "豊田章男（人名, Q418622）\nトヨタ自動車（組織名, Q53268）\n愛知県（地名, Q80434）\n豊田市（地名, Q201117）"
  },
  {
    "id": 2,
    "instruction": "与えられたテキストから固有表現（組織名、人名、地名、固有物名、日付表現、時刻表現、金額表現、割合表現）をすべて抽出し、知識ベース内の該当エンティティにリンクしてください。回答は「固有表現（種類, 知識ベースエンティティ）」の形式で示してください。",
    "input": "福沢諭吉は仕事でアメリカ合衆国に行くために、日本円を米ドルに両替しました。",
    "output": "福沢諭吉（人名, Q355447）\nアメリカ合衆国（地名, Q30）\n日本円（通貨, Q8146）\n米ドル（通貨, Q4917）"
  },
  {
    "id": 3,
    "instruction": "与えられたテキストから固有表現（組織名、人名、地名、固有物名、日付表現、時刻表現、金額表現、割合表現）をすべて抽出し、知識ベース内の該当エンティティにリンクしてください。回答は「固有表現（種類, 知識ベースエンティティ）」の形式で示してください。",
    "input": "Appleは2024年5月7日にAI向けチップとして設計されたM4チップ搭載モデルのiPadを発表しました。",
    "output": "Apple（組織名, Q312)\n2024年5月7日（日付表現, NIL)\nM4チップ（固有物名, Q125823861\niPad（固有物名, Q2796)"
  },
  {
    "id": 4,
    "instruction": "与えられたテキストから固有表現（組織名、人名、地名、固有物名、日付表現、時刻表現、金額表現、割合表現）をすべて抽出し、知識ベース内の該当エンティティにリンクしてください。回答は「固有表現（種類, 知識ベースエンティティ）」の形式で示してください。",
    "input": "日本では4月1日に新年度が始まります。入社式や入学式が行われ、新入生や新入社員が新たな生活をスタートする重要な節目となります。",
    "output": "日本（地名, Q17）\n4月1日（日付表現, Q2510）\n入社式（固有物名, Q11389005）\n入学式（固有物名, Q11388885）\n新入生（固有物名, NIL）\n新入社員（固有物名, Q11255181）"
  },
  {
    "id": 5,
    "instruction": "与えられたテキストから固有表現（組織名、人名、地名、固有物名、日付表現、時刻表現、金額表現、割合表現）をすべて抽出し、知識ベース内の該当エンティティにリンクしてください。回答は「固有表現（種類, 知識ベースエンティティ）」の形式で示してください。",
    "input": "2024.05.10に太陽フレアが連続発生したことにより、通信衛星やGPSに影響を与える恐れがあります。",
    "output": "2024.05.10（日付表現, NIL）\n太陽フレア（固有物名, Q119830）\n通信衛星（固有物名, Q149918）\nGPS（固有物名, Q18822）"
  },
  {
    "id": 6,
    "instruction": "与えられたテキストから固有表現（組織名、人名、地名、固有物名、日付表現、時刻表現、金額表現、割合表現）をすべて抽出し、知識ベース内の該当エンティティにリンクしてください。回答は「固有表現（種類, 知識ベースエンティティ）」の形式で示してください。",
    "input": "東海旅客鉄道の営業利益は東日本旅客鉄道の約1.7倍の5330億円に上ります。",
    "output": "東海旅客鉄道（組織名, Q513679）\n東日本旅客鉄道（組織名, Q499071）\n1.7倍（割合表現, NIL）\n5330億円（金額表現, NIL）"
  },
  {
    "id": 7,
    "instruction": "与えられたテキストから固有表現（組織名、人名、地名、固有物名、日付表現、時刻表現、金額表現、割合表現）をすべて抽出し、知識ベース内の該当エンティティにリンクしてください。回答は「固有表現（種類, 知識ベースエンティティ）」の形式で示してください。",
    "input": "関東大震災は、1923年9月1日11時58分に発生した地震であり、東京を中心に広い範囲で甚大な被害をもたらしました。",
    "output": "東日本大震災（固有物名, Q3742341）\n1923年9月1日（日付表現, NIL）\n11時58分（時刻表現, NIL）\n東京（地名, Q7473516）"
  },
  {
    "id": 8,
    "instruction": "与えられたテキストから固有表現（組織名、人名、地名、固有物名、日付表現、時刻表現、金額表現、割合表現）をすべて抽出し、知識ベース内の該当エンティティにリンクしてください。回答は「固有表現（種類, 知識ベースエンティティ）」の形式で示してください。",
    "input": "日本では「仕事でChatGPT などの生成AI ツールを職場で使用している」と回答した人の割合は38%でした。",
    "output": "日本（地名, Q17）\nChatGPT（固有物名, Q115564437）\n生成AIツール（固有物名, NIL）\n職場（固有物名, Q628858）\n38%（割合表現, NIL）"
  },
  {
    "id": 9,
    "instruction": "与えられたテキストから固有表現（組織名、人名、地名、固有物名、日付表現、時刻表現、金額表現、割合表現）をすべて抽出し、知識ベース内の該当エンティティにリンクしてください。回答は「固有表現（種類, 知識ベースエンティティ）」の形式で示してください。",
    "input": "アジャイル開発の現場でスクラムを取り入れている企業は約半数です。",
    "output": "アジャイル開発（固有物名, Q30232）\n現場（固有物名, NIL）\nスクラム（固有物名, Q460387）\n企業（固有物名, Q3477367）\n約半数（割合表現, NIL）"
  },
  {
    "id": 10,
    "instruction": "与えられたテキストから固有表現（組織名、人名、地名、固有物名、日付表現、時刻表現、金額表現、割合表現）をすべて抽出し、知識ベース内の該当エンティティにリンクしてください。回答は「固有表現（種類, 知識ベースエンティティ）」の形式で示してください。",
    "input": "海外では1000億円超えの資金調達をするスタートアップ企業が続出しています。",
    "output": "海外（地名, Q113641827）\n1000億円超え（金額表現, NIL）\nスタートアップ企業（固有物名, Q129238）"
  }
]

##1-2.データ生成処理の実行

パラメーターをセット

In [ ]:
TEMPERATURE=1 #temperature to use for sampling. 0 means the output is deterministic. Values greater than 1 encourage more diversity
MAX_TOKENS=60000
LOOP_NUM = 10 #1回あたり10件データを生成

In [ ]:
def run_model(PROMPT_TEMPLATE, TEMPERATURE=1, MAX_TOKENS=10000):

    chat_completion = openai.chat.completions.create(
        model="mistralai/Mixtral-8x22B-Instruct-v0.1",
        messages=[{"role": "system", "content": "あなたは日本語の学習データセットを生成する専門家です。"},
         {"role": "user", "content": PROMPT_TEMPLATE}],
        temperature=TEMPERATURE,#temperature to use for sampling. 0 means the output is deterministic. Values greater than 1 encourage more diversity
        top_p=0.9,#0 < top_p ≤ 1 Sample from the set of tokens with highest probability such that sum of probabilies is higher than p. Lower values focus on the most probable tokens.Higher values sample more low-probability tokens
        max_tokens=MAX_TOKENS,
        frequency_penalty=1.3,#-2 ≤ presence_penalty ≤ 2 Positive values penalize new tokens based on how many times they appear in the text so far, increasing the model's likelihood to talk about new topics.
        presence_penalty=1.3 #-2 ≤ presence_penalty ≤ 2 Positive values penalize new tokens based on whether they appear in the text so far, increasing the model's likelihood to talk about new topics.
        )

    output = chat_completion.choices[0].message.content

    return output

In [ ]:
from string import Template
from datetime import datetime
import pandas as pd
import json

df_list = []

### few-shot template ###
formatted_prompt_template = Template(PROMPT_TEMPLATE).substitute(FewShotExamples=FEWSHOT_EXAMPLES)

for i in range(LOOP_NUM):

    ### JSON形式で出力が出てこないケースがあるため、3回までリトライ ###
    for retry_i in range(3):

        ### LLMによるデータ生成処理 ###
        output = run_model(formatted_prompt_template, TEMPERATURE, MAX_TOKENS)
        print(output)

        try:
            #output = json.loads(output)
            df_output = pd.read_json(output)
            df_list.append(df_output)
            print(f"Success parsing JSON")
            break
        except Exception as e:
            print(e)
            print(f"Error parsing JSON: {retry_i}")
            continue


df_final = pd.concat(df_list)

df_final.head()

##1-3.データの保存

In [ ]:
import datetime

# Get the current datetime
now = datetime.datetime.now()

# Format the datetime as a string
timestamp = now.strftime("%Y-%m-%d_%H-%M-%S")

# Create the filename with the timestamp
filename = f"/generated_data_{timestamp}.xlsx"

# Save the DataFrame to an Excel file with the timestamped filename
df_final.to_excel(G_FOLDER + filename, index=False)

print(f"File saved as: {filename}")